In [ ]:
# 安装所需库
!pip install transformers
!pip install sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import os

In [ ]:
# 定义句子嵌入函数（单批处理）
def embed_sentences(sentences, tokenizer, model, mask=True):
    device = next(model.parameters()).device
    inputs = tokenizer(sentences, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.cpu().detach().numpy()

    if mask:
        mask_idx = inputs["input_ids"] == tokenizer.mask_token_id
        mask_idx = mask_idx.cpu()
        embed_temp = []
        for i in range(embedding.shape[0]):
            if mask_idx[i].sum() == 0:
                raise ValueError(f"第 {i} 个句子未找到 [MASK]")
            embed_row = embedding[i, mask_idx[i], :].squeeze()
            if embed_row.ndim > 1:
                embed_row = embed_row.mean(axis=0)
            embed_temp.append(embed_row)
        embedding = np.array(embed_temp)
    else:
        embedding = embedding[:, 0, :].squeeze()

    del inputs, outputs
    torch.cuda.empty_cache()
    return embedding

# 批量处理句子列表
def embed_w_batches(sentences, tokenizer, model, mask=True, batch_size=64):
    torch.cuda.empty_cache()
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
    embeddings = []
    for sentence_batch in chunks(sentences, batch_size):
        embeddings += list(embed_sentences(sentence_batch, tokenizer, model, mask=mask))
    return np.array(embeddings)


In [ ]:
# 设置路径（请根据需要修改）
base_dir = "./project_data"
excel_path = os.path.join(base_dir, "filtered_words.xlsx")
output_dir_main = os.path.join(base_dir, "masked_embeddings")
output_dir_query = os.path.join(output_dir_main, "by_query")
os.makedirs(output_dir_main, exist_ok=True)
os.makedirs(output_dir_query, exist_ok=True)

# 加载模型和分词器
model_name = "IDEA-CCNL/Erlangshen-DeBERTa-v2-710M-Chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModel.from_pretrained(model_name, output_attentions=False).to("cuda" if torch.cuda.is_available() else "cpu")

# 读取词汇表
df = pd.read_excel(excel_path, sheet_name='Sheet1')
df.set_index(df.columns[2], inplace=True)  # 词汇在第 3 列

# 构造模板句
mask_token = tokenizer.mask_token
batch_size = 64
sentences = [f"熟悉我的人说我拥有{mask_token}{mask_token}和{word}的性格" for word in df.index]

# 提取嵌入并保存
vec = embed_w_batches(sentences, tokenizer, model, mask=True, batch_size=batch_size)
vec = vec.reshape((len(df), -1), order='F')
embedding = pd.DataFrame(vec.T, columns=list(df.index))

output_path = os.path.join(output_dir_main, "deberta.csv")
embedding.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"主模板嵌入保存至: {output_path}")

In [ ]:

# 构造多种句式模板
query_templates = {
    "Q1": [f"熟悉我的人说我的性格是{mask_token}{mask_token}和{word}的" for word in df.index],
    "Q2": [f"我的性格是{mask_token}{mask_token}和{word}的" for word in df.index],
    "Q3": [f"我的家人说我的性格是{mask_token}{mask_token}和{word}的" for word in df.index],
    "Q4": [f"讨厌我的人说我的性格是{mask_token}{mask_token}和{word}的" for word in df.index],
    "Q5": [f"我讨厌{mask_token}{mask_token}和{word}的人" for word in df.index],
    "Q6": [f"我希望自己变得更加{mask_token}{mask_token}和{word}" for word in df.index],
    "Q7": [f"她给人的印象总是{mask_token}{mask_token}和{word}" for word in df.index],
    "Q8": [f"他并不总是{word}，有时候也很{mask_token}{mask_token}" for word in df.index],
    "Q9": [f"人们都说他是{mask_token}{mask_token}的，但实际上他很{word}" for word in df.index]
}

# 分别处理每组模板并保存
for query_id, sentences in query_templates.items():
    vec = embed_w_batches(sentences, tokenizer, model, mask=True, batch_size=batch_size)
    vec = vec.reshape((len(df), -1), order='F')
    embedding = pd.DataFrame(vec.T, columns=list(df.index))
    output_path = os.path.join(output_dir_query, f"deberta_{query_id}.csv")
    embedding.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"{query_id} 嵌入保存至: {output_path}")